In [1]:
import idx2numpy
import numpy as np

from mlp import MLP
from bagging import Bagging

In [2]:
def load_data(train_path_images, train_path_labels, test_path_images, test_path_labels):
    X = idx2numpy.convert_from_file(train_path_images)
    y = idx2numpy.convert_from_file(train_path_labels)

    X_test = idx2numpy.convert_from_file(test_path_images)
    Y_test = idx2numpy.convert_from_file(test_path_labels)

    # Convertendo os rótulos para inteiros
    y = y.astype(int)
    y_test = Y_test.astype(int)
    
    # vetorizando os dados
    X = X.reshape(X.shape[0], -1)
    X_test = X_test.reshape(X_test.shape[0], -1)
    
    # normalização dos dados
    X = X / 255.0
    X_test = X_test / 255.0
    
    return X, y, X_test, y_test

In [3]:
# Carregando a base MNIST
train_path_images = 'dataset/train/train-images.idx3-ubyte'
train_path_labels = 'dataset/train/train-labels.idx1-ubyte'

test_path_images = 'dataset/test/t10k-images.idx3-ubyte'
test_path_labels = 'dataset/test/t10k-labels.idx1-ubyte'

X, y, X_test, Y_test = load_data(train_path_images, train_path_labels, test_path_images, test_path_labels)

print(f'Dados de treino: \n{X} \nResultados de treino: {y} \nDados de teste: \n{X_test} \nResultados de teste: {Y_test}')

Dados de treino: 
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 
Resultados de treino: [5 0 4 ... 5 6 8] 
Dados de teste: 
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 
Resultados de teste: [7 2 1 ... 4 5 6]


In [4]:
# Inicialização da classe que irá diversificar os dados
bagging = Bagging()

In [5]:
# Treinamento do comitê de perceptrons

num_committee = 5
predictions_matrix = list()

for i in range(num_committee):
    # Aplicando o bagging para embaralhar o dataset de treino com reposição
    # sample, sample_labels = bagging.subsample(X, y)
    print(f'------------------------------- Modelo {i+1} -------------------------------')
    
    if i == 0:
        init_num_neur = 64
    elif i == 1:
        init_num_neur = 72
    elif i == 2:
        init_num_neur = 80
    elif i == 3:
        init_num_neur = 88
    elif i == 4:
        init_num_neur = 96
    elif i > 4:
        pass

    print(f'[INFO] Numero de neurônios: {init_num_neur}')
    # carregando o modelo
    model = MLP(
        X=X,
        y=y,
        X_val=X_test,
        y_val=Y_test,
        epochs=100,
        num_neurons_hidden_layer=init_num_neur,
        seed=False,
        save_predictions=True,
        verbose=True
    )
    
    # treinando o modelo
    model.fit()
    
    # armazenando os valores de inferencia dos modelos a cada n (valor padrao é 50 epocas) epocas em cima dos dados de teste
    predictions_matrix.append(model.predictions_matrix[0])

------------------------------- Modelo 1 -------------------------------
[INFO] Numero de neurônios: 64
[TREINAMENTO] - Numero de neuronios: 64 - Epoca 10 - Loss: 0.0420315166661629 - Predições: [7 2 1 ... 4 8 6] - Acurácia: 0.7375 - Taxa de Erro - 0.2625 2023-09-05 16:17:59.240273
[TREINAMENTO] - Numero de neuronios: 64 - Epoca 20 - Loss: 0.02026711291083844 - Predições: [7 2 1 ... 4 8 6] - Acurácia: 0.8713 - Taxa de Erro - 0.1287 2023-09-05 16:18:07.769358
[TREINAMENTO] - Numero de neuronios: 64 - Epoca 30 - Loss: 0.015515250078284061 - Predições: [7 2 1 ... 4 5 6] - Acurácia: 0.9045 - Taxa de Erro - 0.0955 2023-09-05 16:18:16.007685
[TREINAMENTO] - Numero de neuronios: 64 - Epoca 40 - Loss: 0.013130140046954765 - Predições: [7 2 1 ... 4 5 6] - Acurácia: 0.9166 - Taxa de Erro - 0.0834 2023-09-05 16:18:23.771176
[TREINAMENTO] - Numero de neuronios: 64 - Epoca 50 - Loss: 0.011680586444506926 - Predições: [7 2 1 ... 4 5 6] - Acurácia: 0.9262 - Taxa de Erro - 0.0738 2023-09-05 16:18:30.8

KeyboardInterrupt: 

In [ ]:
# Acuracia do comitê

# montando a matriz com todos os valores preditos por cada um dos modelos ao longo das epocas de treinamento
committee_predicts_ = np.stack(predictions_matrix, axis=1)

# # Aplicar votação majoritária em cada coluna da matriz committee_predicts_
final_predictions_by_epoch_ = np.apply_along_axis(bagging.majority_vote, axis=1, arr=committee_predicts_)


# Calcular a acurácia para o vetor com as predições mais votadas dentre todos os modelos
accuracy_ = np.mean(final_predictions_by_epoch_ == Y_test)
print('-----------------------------------------------------------------------------------------------------------------')
print(f'[INFERENCIA DO COMITÊ] Predições (voto majoritario): {final_predictions_by_epoch_} - Acurácia do Comitê na melhor época: {accuracy_:.4f}')
print('-----------------------------------------------------------------------------------------------------------------')
# -------------------------------------------------------------------------------------- #

-----------------------------------------------------------------------------------------------------------------
[INFERENCIA DO COMITÊ] Predições (voto majoritario): [7 2 1 ... 4 5 6] - Acurácia do Comitê na melhor época: 0.9466
-----------------------------------------------------------------------------------------------------------------
